# 11. 使用Apriori算法进行关联分析

从大规模数据集中寻找物品间的隐含关系被称为关联分析(association analysis)或者关联规则学习(association rule learning)。

大规模数据的关系可以有两种形式：频繁项集和关联规则  
- 频繁项集(frequent item sets)：经常出现在一起的物品的集合
- 关联规则(association ruls)：暗示两种物品之间可能存在很强的关系

**名词解释**  
- 频繁模式(frequent patterns)：是一种在数据集中频繁出现的模式（例如 项集，子序列，子结构）。例如牛奶和面包在一组交易数据集合中频繁的出现，那么称牛奶和面包是频繁项集；子序列，例如先买电脑，之后买相机，之后买存储卡，若它在购买历史中频繁出现，那么它是一个频繁序列模式；子结构可以指不同的结构形式，如子图、子树或子格，它们可以与项集或子序列组合在一起。如果一个子结构频繁出现，它被称为(频繁的)结构化模式。


- 支持度(support):数据集中包含该项集的记录所占的比例。从下面的例子中可以看出{豆奶}的支持度为4/5，{豆奶，尿布}的支持度为3/5。支持度是针对项集来说的，因此可以定义一个最小支持度，只保留满足最小支持度的项集。


- 置信度(confidence):置信度是针对一条诸如{尿布}->{葡萄酒}的关联规则来定义的。这条规则的可信度定义为 **支持度({尿布，葡萄酒})/支持度({尿布})** 。以下面的数据为例，support({尿布，葡萄酒})=3/5， support({尿布})=4/5，所以confidence(尿布->葡萄酒)=3/4=0.75。这意味着对于包含“尿布”的所有记录，我们的规则对其75%的记录都适用。


| 交易编号| 商品
| ---| ---|
| 0 | 豆奶，莴苣 |
| 1 | 莴苣，尿布，葡萄酒，甜菜|
| 2 | 豆奶， 尿布，葡萄酒，橙汁 | 
| 3 | 莴苣，豆奶，尿布，葡萄酒 |
| 4 | 莴苣，豆奶，尿布，橙汁 | 

## 11.1. Apriori
一个拥有N个物品的集合共有$2^N-1$种项集组合，所需的计算时间非常长。Apriori是为了解决这个问题而提出的算法。

Apriori的原理是如果某个项集是频繁的，那么它的所有子集也是频繁的。如果一个项集是非频繁的，那么它的超集也是非频繁的。例如计算出项集{2,3}是非频繁的，那么就知道{0,2,3},{1,2,3},{0,1,2,3}也是非频繁的，就不在需要计算这些项集的支持度。使用该原理就可以避免项集数目的指数增长。

In [44]:
import pandas as pd
import numpy as np

In [25]:
# 遍历数据集中的每一个交易，将单个的交易项组成一个集合（frozenset）
# 这里需要注意的一点是：map在python2中返回一个列表，在python3中返回一个迭代器，因此这里需要将返回值转成list
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))


'''
    参数： D:  数据集
          Ck: 候选集列表
          minSupport: 最小支持度
          
    返回值： retList: 满足最小支持度的项集列表
            supportData：满足最小支持度的以项集为key，以支持度为value的字典
'''
def scanD(D, Ck, minSupport):
    ssCnt = {}
    # 对于数据集中的每一条记录
    for tid in D:
        # 对于候选项集中的每一项
        for can in Ck:
            # 如果该项集是记录的一个子集，则将字典中的记录+1
            if can.issubset(tid):
                if not ssCnt.get(can): ssCnt[can] = 1
                else: ssCnt[can] += 1
                    
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        # 如果支持度大于最小支持度，则加入结果列表中
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [26]:
dataSet = [[1,3,4], [2,3,5], [1,2,3,5], [2,5]]
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [27]:
D = list(map(set, dataSet))

In [28]:
scanD(D, C1, 0.5)

([frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 {frozenset({4}): 0.25,
  frozenset({5}): 0.75,
  frozenset({2}): 0.75,
  frozenset({3}): 0.75,
  frozenset({1}): 0.5})

In [11]:
len(list(D))

0

## 11.2. Apriori算法


伪代码如下：
```
当集合中项的个数大于0时
    构建一个k个项组成的候选项集的列表
    检查数据以确认每个项集都是频繁的
    保留频繁项集并构建k+1项组成的候选集的列表
```

In [35]:
'''
    参数： Lk: 频繁项集列表
           k: 将要生成的项集元素个数
           
    k-2 保证了相对于上一次的项集，这次生成的项集会多一位数字
           
    以 Lk = {0} {1} {2}, k=2 举例:
        k-2 = 0
        L1 = []
        L2 = []
        retList = [{0,1}, {0,2}, {1,2}]
'''
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

'''

'''
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [37]:
L, supportData = apriori(dataSet)

In [38]:
L

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({3, 5}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3})],
 [frozenset({2, 3, 5})],
 []]

In [42]:
supportData

{frozenset({5}): 0.75,
 frozenset({3}): 0.75,
 frozenset({2, 3, 5}): 0.5,
 frozenset({1, 2}): 0.25,
 frozenset({1, 5}): 0.25,
 frozenset({3, 5}): 0.5,
 frozenset({4}): 0.25,
 frozenset({2, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({1}): 0.5,
 frozenset({1, 3}): 0.5,
 frozenset({2}): 0.75}

## 11.3. 从频繁项集中挖掘关联规则

In [39]:
'''
    作用: 主函数，调用下面两个函数
    参数: L: 频繁项集列表
         supportData: 满足最小支持度的以项集为key，以支持度为value的字典
         minConfi: 最小可信度阈值
    返回值: bigRuleList: 包含可信度的规则列表
'''
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    # 只获取两个或更多元素的集合，因为无法从单个元素项集中构建关联规则
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            # 如果项集中只有两个元素则直接通过calcConf计算
            if i>1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
                
    return bigRuleList

'''
    作用： 对规则进行评估
    参数: freqSet: 频繁项集
         H: 只包含单个元素的集合列表
         supportData:
         br1:
         minconf:
         
    返回值: prunedH: 满足最小可信度的规则列表
    
'''
def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(freqSet-conseq,"--->",conseq, "conf:",conf)
                br1.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH
            
'''
    作用: 生成候选规则集合
    参数: freqSet:
         H:
         supportData:
         br1:
         minconf:
'''
def rulesFromConseq(freqSet, H, supportData, br1, minConf):
    m = len(H[0])
    if (len(freqSet) > (m+1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

In [40]:
generateRules(L, supportData)

frozenset({1}) ---> frozenset({3}) conf: 1.0
frozenset({5}) ---> frozenset({2}) conf: 1.0
frozenset({2}) ---> frozenset({5}) conf: 1.0


[(frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0)]

In [41]:
generateRules(L, supportData, 0.5)

frozenset({5}) ---> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) ---> frozenset({5}) conf: 0.6666666666666666
frozenset({3}) ---> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) ---> frozenset({3}) conf: 1.0
frozenset({5}) ---> frozenset({2}) conf: 1.0
frozenset({2}) ---> frozenset({5}) conf: 1.0
frozenset({3}) ---> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) ---> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) ---> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) ---> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) ---> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

## 11.4. 发现毒蘑菇的相似特征

In [55]:
#mushroomdf = pd.read_csv("mushroom.dat", sep=' ', header=None)
mushroomdf = [line.split() for line in open('mushroom.dat').readlines()]
L, supportData = apriori(np.array(mushroomdf), minSupport=0.3)

In [54]:
for item in L[1]:
    if item.intersection('2.0'): print(item)

In [50]:
print(L[1])

[frozenset({2.0, 76.0}), frozenset({3.0, 86.0}), frozenset({59.0, 53.0}), frozenset({59.0, 76.0}), frozenset({116.0, 85.0}), frozenset({90.0, 67.0}), frozenset({94.0, 86.0}), frozenset({39.0, 23.0}), frozenset({85.0, 110.0}), frozenset({36.0, 63.0}), frozenset({36.0, 93.0}), frozenset({59.0, 39.0}), frozenset({10.0, 34.0}), frozenset({56.0, 85.0}), frozenset({90.0, 38.0}), frozenset({53.0, 94.0}), frozenset({85.0, 93.0}), frozenset({86.0, 63.0}), frozenset({85.0, 94.0}), frozenset({36.0, 110.0}), frozenset({90.0, 85.0}), frozenset({93.0, 39.0}), frozenset({63.0, 39.0}), frozenset({116.0, 36.0}), frozenset({24.0, 34.0}), frozenset({1.0, 85.0}), frozenset({67.0, 93.0}), frozenset({3.0, 85.0}), frozenset({34.0, 59.0}), frozenset({2.0, 86.0}), frozenset({90.0, 39.0}), frozenset({34.0, 3.0}), frozenset({10.0, 86.0}), frozenset({59.0, 86.0}), frozenset({2.0, 59.0}), frozenset({10.0, 36.0}), frozenset({24.0, 85.0}), frozenset({90.0, 6.0}), frozenset({85.0, 63.0}), frozenset({34.0, 85.0}), fro